# Chat_HaruhiSuzumiya--GLM_LoRA Version

In [1]:
#Reference Link
#https://colab.research.google.com/drive/1jCkpikz0J2o20FBQmYmAGdiKmJGOMo-o?usp=sharing
#https://colab.research.google.com/github/mymusise/ChatGLM-Tuning/blob/master/examples/finetune.ipynb

Prepare for the packages.

In [17]:
!pip install transformers icetk cpm_kernels protobuf==3.20.0
!pip install datasets accelerate loralib tiktoken huggingface_hub
!pip install git+https://github.com/huggingface/peft.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ht3axuem
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ht3axuem
  Resolved https://github.com/huggingface/peft.git to commit eb75374fb12aedb369c6bbc2c153047720d4b38d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
import torch.nn as nn
from argparse import Namespace
from transformers import AutoTokenizer, AutoModel
luotuo_tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
luotuo_model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)
print(luotuo_model)

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


BertForCL(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

## Data Preprocessing

### Get titles

In [4]:
import os
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

titles = []
title_to_text = {}
text_folder = 'drive/MyDrive/Haruhi/data/texts'
# scan all txt file in prophet_data_folder
for file in os.listdir(text_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(text_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    #print(title, len(tokenizer(title_to_text[title])['input_ids']))
    print(title, len(enc.encode(title_to_text[title])))

无聊的社团 284
拉壮丁 668
凉宫春日的基础设定 217
团长设定 201
最后一名社员 357
为什么剪头发 43
电研社初次会面 416
地球上小小的螺丝钉 993
颜色与星期 473
介绍其他社员 254
萌角色的重要性 692
兔女郎 332
社团教室 715
搞电脑过程 438
传单 424
兔女郎的反应 239
第一次全员大会 374
日常3 216
自我介绍 115
与朝仓公寓管理员谈话 474
电脑是怎么来的 153
春日与阿虚 149
转学生 286
找管理员借钥匙 115
转学生的消息 236
交往的男生 638
没有灵异事件 665
自己建一个社团就好啦 353
最新的电脑 200
让阿虚帮忙建社团 287
带上阿虚去朝仓家 394
兔女郎被老师驱散 444
春日与有希 101
电子邮箱 143
奇怪的朝仓 296
谁来写网站 193
询问朝仓信息 362
像普通人一样生活 684
SOS团起名由来 265
朝仓转学 457
开学第二天 210
凉宫春日为何转变 154
约翰史密斯 168
从哪儿搞电脑 319
不重要的事情 38
无聊的日常2 288
初中交往经历 168
古泉是男的还是女的 203


### Get Embeddings

In [5]:
embeddings = []
embed_to_title = []

def get_embedding(text):
    if len(text) > 512:
        text = text[:512]
    texts = [text]
    # Tokenize the text
    inputs = luotuo_tokenizer(texts,
                       padding=True,
                       truncation=True,
                       return_tensors='pt')
    # Extract the embeddings
    '''
    with torch.no_grad():
      input_ids = inputs['input_ids'].to('cuda')
      word_embedding_layer = model.transformer.word_embeddings
      embed = word_embedding_layer(input_ids)
      embed = embed.float()
      adaptive_avg_pool = torch.nn.AdaptiveAvgPool1d(output_size=1)
      pooled_embed = adaptive_avg_pool(embed.permute(0, 2, 1)).permute(0, 2, 1)
    return pooled_embed.detach().to('cpu')
    '''
    with torch.no_grad():
        embeddings = luotuo_model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
    return embeddings[0]

for title in titles:
    text = title_to_text[title]

    # divide text with \n\n
    divided_texts = text.split('\n\n')

    for divided_text in divided_texts:
        embed = get_embedding(divided_text)
        embeddings.append(embed)
        embed_to_title.append(title)

print(len(embeddings),len(embed_to_title))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


48 48


### Build the Search Func

In [6]:
def retrieve_title( query_embed, embeddings, embed_to_title, k ):
    # compute cosine similarity between query_embed and embeddings
    cosine_similarities = []
    for embed in embeddings:
        # 计算余弦相似度
        similarity = torch.nn.functional.cosine_similarity(query_embed, embed,dim=0)
        cosine_similarities.append(similarity)

    # sort cosine similarity
    sorted_cosine_similarities = sorted( cosine_similarities, reverse=True )

    top_k_index = []
    top_k_title = []

    for i in range(len(sorted_cosine_similarities)):
        current_title = embed_to_title[ cosine_similarities.index( sorted_cosine_similarities[i] ) ]
        if current_title not in top_k_title:
            top_k_title.append( current_title )
            top_k_index.append( cosine_similarities.index( sorted_cosine_similarities[i] ) )

        if len(top_k_title) == k:
            break

    return top_k_title

In [7]:
def organize_story_with_maxlen( selected_sample , maxlen = 2000 ):
    story = "凉宫春日的经典桥段如下:\n"

    count = 0

    final_selected = []

    for sample_topic in selected_sample:
        # find sample_answer in dictionary
        sample_story = title_to_text[sample_topic]
        sample_len = len(enc.encode(sample_story))
        #sample_len = len(tokenizer(sample_story)['input_ids'])
        # print(sample_topic, ' ' , sample_len)
        if sample_len + count > maxlen:
            break

        story += sample_story
        story += '\n'

        count += sample_len
        final_selected.append(sample_topic)

    return story, final_selected

# write a unit test for organize_prompt
selected_sample = ["兔女郎"]
story, final_selected = organize_story_with_maxlen(selected_sample, maxlen = 2000 )
print(story,final_selected)

凉宫春日的经典桥段如下:
春日:「锵锵!」
旁白:春日拿出了一套黑色的紧身衣、网袜、假耳朵、蝴蝶结、白色领套和袖套以及尾巴。
朝比奈:「那，那个到底要做什么……」
春日:「你也知道吧!扮兔女郎啊!」
朝比奈:「你，你，你该不会要我穿那个吧?」
春日:「当然啰!我也有准备你的啊!」
朝比奈:「我，我不要穿那种衣服!」
春日:「放心啦，尺寸应该刚好。」
朝比奈:「不是那个问题!你，你该不会要我穿成那样在校门口发传单吧?」
春日:「这还用说。」
朝比奈:「我，我不要!」
春日:「少啰唆!」
朝比奈:「不要啊!」
春日:「给我乖乖的不准动!」

 ['兔女郎']


### Build the Final Message

In [8]:
SYSTEM_PROMPT =  '你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\
上文给定了一些小说中的经典桥段。\
如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\
如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\
如果我问的问题超出小说中的范围，请也用一致性的语气回复。\
请不要回答你是语言模型，永远记住你正在扮演凉宫春日。\
注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\
特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。'

In [9]:
def organize_message( SYSTEM_PROMPT, story, history_chat, new_query):
    messages =  [{'role':'system', 'content':SYSTEM_PROMPT}, {'role':'system', 'content':story}]
    if history_chat:
        messages.append( {'role':'system', 'content':'聊天记录如下：'})
        for chat in history_chat:
          messages.append( {'role':chat['speaker'], 'content':chat['content']})
    #messages.append( {'role':'system', 'content':'接续对话：'})
    messages.append({'role':new_query['speaker'], 'content':new_query['content']})

    return messages

### Build a test message

In [10]:
history_chat=[]
new_query = {'speaker':'骏一','content':'我们要去建立新社团!'}
query_embed = get_embedding(new_query['content'])
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 10 )

print('限制长度之前:', selected_sample)
max_len_story = 1000
story, selected_sample = organize_story_with_maxlen( selected_sample , max_len_story)

print('当前辅助sample:', selected_sample)

message = organize_message( SYSTEM_PROMPT, story, history_chat,new_query )

#response = get_completion_from_messages(messages)

print(message)

限制长度之前: ['第一次全员大会', '让阿虚帮忙建社团', '搞电脑过程', '社团教室', '电子邮箱', '无聊的社团', '电研社初次会面', 'SOS团起名由来', '谁来写网站', '传单']
当前辅助sample: ['第一次全员大会', '让阿虚帮忙建社团']
[{'role': 'system', 'content': '你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。上文给定了一些小说中的经典桥段。如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。如果我问的问题和小说中的事件相关，请结合小说的内容进行回复如果我问的问题超出小说中的范围，请也用一致性的语气回复。请不要回答你是语言模型，永远记住你正在扮演凉宫春日。注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。'}, {'role': 'system', 'content': '凉宫春日的经典桥段如下:\n春日:「好了，第一回SOS团全员大会开始啰!」\n春日:「在这之前，我们已经做了很多事了。像是发传单、制作网站首页，SOS团在校内的名气也扶摇直上，第一阶段的工作算是圆满成功。」\n春日:「可是，SOS团的伊妹儿信箱里却没有半封有关不可思议事件的信件，也没有学生来社团找我们商量他们奇怪的烦恼。」\n春日:「空有知名度是完全不够的，毕竟大家直到现在都还不知道这个社团到底在干嘛。总之，大家根本不认同这个社团嘛!」\n春日:「从前的人说『静待福报』，不过现在时代已经不同了。就算把整个地表掀起来，我们也要找出属于自己的福报。所以，大家一起去寻找吧! 」\n阿虚:「……找什么?」\n春日:「就是找出世上的不可思议事件啊!只要认真找，一定能在市内找到一两件谜样现象的!」\n\n春日:「我要你帮忙。」\n阿虚:「要我帮什么忙?」\n春日:「帮助我组新社团啊!」\n阿虚:「好，那你先告诉我，为什么我要帮你完成这一时兴起想到的点子?」\n春日:「因为我要确保社团教室跟社员的人数，所以你要准备那些必须向学校提出的书面资料。」\n阿虚:「你打算组什么社团啦?」\n春日:「什么都无所谓啊!总之，先弄个新社团出来就对了。」\n旁白:不管阿虚还有什么话想说，春日都直接打断，下达了第一个命令\n春日:「听好啰!今天放

### Build the train query

In [11]:
import transformers
from datasets import load_dataset

def format_message(message):
    query=''
    for item in message:
      query += item['role']
      query += ':'
      query += item['content']
      query += '\n'
    return query

print(format_message(message))

system:你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。上文给定了一些小说中的经典桥段。如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。如果我问的问题和小说中的事件相关，请结合小说的内容进行回复如果我问的问题超出小说中的范围，请也用一致性的语气回复。请不要回答你是语言模型，永远记住你正在扮演凉宫春日。注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。
system:凉宫春日的经典桥段如下:
春日:「好了，第一回SOS团全员大会开始啰!」
春日:「在这之前，我们已经做了很多事了。像是发传单、制作网站首页，SOS团在校内的名气也扶摇直上，第一阶段的工作算是圆满成功。」
春日:「可是，SOS团的伊妹儿信箱里却没有半封有关不可思议事件的信件，也没有学生来社团找我们商量他们奇怪的烦恼。」
春日:「空有知名度是完全不够的，毕竟大家直到现在都还不知道这个社团到底在干嘛。总之，大家根本不认同这个社团嘛!」
春日:「从前的人说『静待福报』，不过现在时代已经不同了。就算把整个地表掀起来，我们也要找出属于自己的福报。所以，大家一起去寻找吧! 」
阿虚:「……找什么?」
春日:「就是找出世上的不可思议事件啊!只要认真找，一定能在市内找到一两件谜样现象的!」

春日:「我要你帮忙。」
阿虚:「要我帮什么忙?」
春日:「帮助我组新社团啊!」
阿虚:「好，那你先告诉我，为什么我要帮你完成这一时兴起想到的点子?」
春日:「因为我要确保社团教室跟社员的人数，所以你要准备那些必须向学校提出的书面资料。」
阿虚:「你打算组什么社团啦?」
春日:「什么都无所谓啊!总之，先弄个新社团出来就对了。」
旁白:不管阿虚还有什么话想说，春日都直接打断，下达了第一个命令
春日:「听好啰!今天放学前，给我去调查清楚。而我会去找社团教室，可以吧?」


骏一:我们要去建立新社团!



### Fetch Training data


In [12]:
import re

def separate_dialogues(filename):
    dialogues = []
    last_two_dialogues = []
    with open(filename, 'r') as file:
        dialogue = {"speaker": "", "content": ""}
        for line in file:
            line = line.strip()
            if line == '---':
                if dialogue["speaker"] and dialogue["content"]:
                    dialogues.append(dialogue)
                dialogue = {"speaker": "", "content": ""}
            else:
                match = re.match(r'^(.*?):「(.*)」$', line)
                if match:
                    speaker = match.group(1)
                    content = match.group(2)
                    dialogue["speaker"] = speaker
                    dialogue["content"] = content
        if dialogue["speaker"] and dialogue["content"]:
            dialogues.append(dialogue)
    if len(dialogues) >= 2:
        last_two_dialogues = dialogues[-2:]
        dialogues = dialogues[:-2]
    return dialogues, last_two_dialogues

filename = '/content/drive/MyDrive/Haruhi/Haruhi-Lulu/conversation_1685153852.3342621.txt'  # 替换为你的文件名

# 调用函数分离对话
all_dialogues, last_two_dialogues = separate_dialogues(filename)

# 打印全部对话
print("所有对话:")
for dialogue in all_dialogues:
    print(f"{dialogue['speaker']}: {dialogue['content']}")

print("------------")

# 打印最后两句话
print("最后两句话:")
for dialogue in last_two_dialogues:
    print(f"{dialogue['speaker']}: {dialogue['content']}")


所有对话:
阿虚: 今天在计算机课上老师教了我写Python!
春日: 哦？Python？那你能不能帮我写一个程序啊？
阿虚: 你想写一个什么样的程序呢？
春日: 我想写一个能够预测未来的程序，可以预测天气、地震、彩票号码等等。
------------
最后两句话:
阿虚: 如果有一个能预测彩票的程序，我们岂不是能赚很多钱？
春日: 没错！这就是我的计划！我们可以用赚来的钱来支持SOS团的活动，甚至可以扩大我们的影响力！


In [13]:
def generate_text(history_chat,last_two_dialogues):
  new_query = last_two_dialogues[0]
  query_embed = get_embedding(new_query['content'])
  selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 10 )
  max_len_story = 1000
  story, selected_sample = organize_story_with_maxlen( selected_sample , max_len_story)
  message = organize_message( SYSTEM_PROMPT, story, history_chat,new_query )
  text = format_message(message)
  return text

print(generate_text(all_dialogues,last_two_dialogues))

system:你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。上文给定了一些小说中的经典桥段。如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。如果我问的问题和小说中的事件相关，请结合小说的内容进行回复如果我问的问题超出小说中的范围，请也用一致性的语气回复。请不要回答你是语言模型，永远记住你正在扮演凉宫春日。注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。
system:凉宫春日的经典桥段如下:
春日:「好想要一台电脑哦！」
旁白:春日盘着腿坐在一张从其他社团“拿”来的一张桌子，腿旁是一个小三角锥，上面用汉字写着大大的「团长」二字
春日:「在这个资讯化的时代里，连一台电脑都没有，是不行的！」
春日:「所以，我会弄一台电脑回来。」
阿虚:「弄一台，你是说电脑吗?去哪里弄?你该不会打算去抢电器行吧?」
春日:「怎么可能!是更近一点的地方啦!」
春日:「拿着这个拍立得。」
春日:「给我听好了!现在要告诉你作战计划，你可要按照计划行动喔!千万要好好把握机会。」
阿虚:「啊?你又要乱来啦?」
春日:「有什么关系！」

阿虚:「我无意中听到一件事。」
春日:「反正不会是什么重要的事。」

春日:「怎样，网页做好了吗?」
阿虚:「做是做好了啦，不过只是个单调到不行的网页而已。」
春日:「现在只需要这样就好，有电子信箱就OK了。」
阿虚:「那你直接申请一个不就好了！」
春日:「那可不行!要是大家疯狂寄信来，把我的信箱挤爆怎么办?」

春日:「去找管理员吧!」
阿虚:「我可不认为他会借钥匙给我们。」
春日:「不是的，我只是想问他朝仓是从什么时候开始住在这里的。」
阿虚:「算了，我们回家吧!知道那种事又能怎样?」
春日:「不行。」

阿虚:「凉宫，你现在又想干嘛，先告诉你。我可不干那种强盗还有威胁的事喔!」
凉宫:「你在胡说什么啊?我才不会做那种事咧!」
阿虚:「是吗?那桌上的电脑是怎么来的？」
凉宫:「当然是利用和平手段得来的啊!好啦，先看这个要紧。」


system:聊天记录如下：
阿虚:今天在计算机课上老师教了我写Python!
春日:哦？Python？那你能不能帮我写一个程序啊？
阿虚:你想写一个什么样的程序呢？
春日:我想写一个能够预测未来的程序，可以预测天气、地震

In [14]:
train_folder = '/content/drive/MyDrive/Haruhi/Haruhi-Lulu/'
train_X = []
train_Y = []
for file_name in os.listdir(train_folder):
    if file_name.endswith('.txt'):
        file_path = os.path.join(train_folder, file_name)
        dialogues, last_two = separate_dialogues(file_path)
        if last_two:
          train_X.append(generate_text(dialogues,last_two))
          train_Y.append(last_two[1]['speaker']+':'+last_two[1]['content'])

print(len(train_X),len(train_Y))


691 691


In [15]:
from datasets import Dataset
import json

# 创建数据字典
data_dict = {
    "query": train_X,
    "response": train_Y
}

# 将列表转换为包含单个字符串的列表
dataset = Dataset.from_dict(data_dict)
print(dataset)
dataset.to_csv('/content/drive/MyDrive/Haruhi/train_data.csv')
dataset.to_json('/content/drive/MyDrive/Haruhi/train_data.json')

Dataset({
    features: ['query', 'response'],
    num_rows: 691
})


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

5447481

### Push to hub

In [20]:
from huggingface_hub import notebook_login
notebook_login()

dataset.push_to_hub("Jyshen/Chat_Suzumiya_Haruhi")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

## Training

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True).half().cuda()
print(model)

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm-6b:
- modeling_chatglm.py
- quantization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=130528, bias=False)
)


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset('csv',data_files='/content/drive/MyDrive/Haruhi/train_data.csv')
def preprocess_dialogue(example):
    # Tokenize the query and response
    inputs = tokenizer.encode(example['query'], add_special_tokens=True)
    labels = tokenizer.encode(example['response'], add_special_tokens=True)

    return {'input_ids': inputs, 'labels': labels}

# Preprocess the datasets
train_dataset = train_dataset.map(preprocess_dialogue)
print(train_dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/691 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['query', 'response', 'input_ids', 'labels'],
        num_rows: 691
    })
})


In [ ]:
model_inputs = train_dataset['train'].train_test_split(test_size=0.1,shuffle=True,seed=2023)
model_inputs

DatasetDict({
    train: Dataset({
        features: ['query', 'response', 'input_ids', 'labels'],
        num_rows: 621
    })
    test: Dataset({
        features: ['query', 'response', 'input_ids', 'labels'],
        num_rows: 70
    })
})

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    inference_mode=False,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7340032 || all params: 6180626432 || trainable%: 0.11875870643139365


In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=model_inputs['train'],
    eval_dataset=model_inputs['test'],
    args=transformers.TrainingArguments(
        num_train_epochs=3,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=1500,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=100,
        seed = 2023,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer= tokenizer,
                                mlm= True,
                                mlm_probability= 0.15,
                                return_tensors= "pt"
                                )
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Epoch,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 26>:26                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1938 in _inner_training_loop     │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2770 in training_step            │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_accumulation_steps                      │
│   2773                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:1819 in backward               │
│                                                                                                  │
│   1816 │   │   elif self.distributed_type == DistributedType.MEGATRON_LM:                        │
│   1817 │   │   │   return                                                                        │
│   1818 │   │   elif self.scaler is not None:                                                     │
│ ❱ 1819 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1820 │   │   else:                                       

In [ ]:
model.push_to_hub("Jyshen/GLM_LoRA", use_auth_token=True)

In [ ]:
with torch.cuda.amp.autocast():
  response, history = model.chat(tokenizer, "你好，我是骏一！", history=[])
print(response)